# University of Glasgow - Medical Physics MSc
## Radiotherapy Computing Tutorial & Computing/Planning/Brachytherapy Assignment

This [Jupyter notebook](https://jupyter.org/) will act as the Radiotherapy Computing tutorial, and as the assignment for computing/planning/brachytherapy.    

This notebook will be running in an online executable environment hosted by [mybinder.org](https://mybinder.org/). To run code cells you can use `Control + Enter`. The code at any point depends what has been run before it chronologically, not necessarily through this document top down. To rerun the entire notebook, at the top of this page click the "Fast Forward" button (to the left of Download). For any issues with the environment please get in touch with [Alasdair Rutherford](mailto:alasdair.rutherford@ggc.scot.nhs.uk)

In the resources folder we have a single field radiotherapy plan object (`RP.singlefield.dcm`), a multiple field radiotherapy plan object (`RP.mutlifield.dcm`), and tables of [percentage depth doses](./resources/PDDs.pdf) and [output factors](./resources/OutputFactors.pdf).

###  Example - Interacting with External Beam Plans

For this plan, the "patient" is a big block of water, the CT dataset is actually blank and the "patient" is created from the structure set. To start interacting with the patient, lets import some packages. We will be using [pydicom](https://pydicom.github.io/) to interact with the DICOM object.

In [34]:
import pydicom as pd

Now the package is imported, we will load the plan and show the information:

In [35]:
ds_rtplan = pd.dcmread("resources/RP.singlefield.dcm")

In [36]:
ds_rtplan

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 192
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: RT Plan Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.246.352.221.5446671153178189143.3115329015485527700
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.246.352.70.2.1.160.3
(0002, 0013) Implementation Version Name         SH: 'DCIE 16.1'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20220715'
(0008, 0013) Instance Creation Time              TM: '102008'
(0008, 0016) SOP Class UID                       UI: RT Plan Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.221.5446671153178189143.3115329015485527700
(0008, 0

Next lets check for the number of beams and load them

In [37]:
print("We can count the number of Beam Sequences: " +  str(len((ds_rtplan.BeamSequence))))

We can count the number of Beam Sequences: 1


In our example plan, there is only one beam - so let's see the relevant detail. 

**Note**: for details on the units involved, see:  
[Table C.8-46: RT Prescription Module Attributes](https://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_c.8.8.10.html)  
[Table C.8-50: RT Beams Module Attributes](https://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_C.8.8.21.html)

In [38]:
print(ds_rtplan.BeamSequence)

[(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 1090) Manufacturer's Model Name           LO: 'TDS'
(3002, 0050)  Primary Fluence Mode Sequence  1 item(s) ---- 
   (3002, 0051) Fluence Mode                        CS: 'STANDARD'
   ---------
(300a, 00b2) Treatment Machine Name              SH: 'TRF_18'
(300a, 00b3) Primary Dosimeter Unit              CS: 'MU'
(300a, 00b4) Source-Axis Distance                DS: '1000.0'
(300a, 00b6)  Beam Limiting Device Sequence  2 item(s) ---- 
   (300a, 00b8) RT Beam Limiting Device Type        CS: 'X'
   (300a, 00bc) Number of Leaf/Jaw Pairs            IS: '1'
   ---------
   (300a, 00b8) RT Beam Limiting Device Type        CS: 'Y'
   (300a, 00bc) Number of Leaf/Jaw Pairs            IS: '1'
   ---------
(300a, 00c0) Beam Number                         IS: '1'
(300a, 00c2) Beam Name                           LO: 'Field 1'
(300a, 00c4) Beam Type                           CS: 'STATIC'
(300a, 00c6) Radiation Type   

At this point, with the above information, it'd be useful to draw a diagram of the setup.

Hint: if you are unsure of the name of a specific tag in pydicom you can use the following syntax to see related results `ds_rtplan.dir("searchterm")`, example below.

In [39]:
ds_rtplan.dir("Plan")

['PlanIntent',
 'RTPlanDate',
 'RTPlanGeometry',
 'RTPlanLabel',
 'RTPlanName',
 'RTPlanTime']

For our patient, we can see the pertinent information.

For this patient:

| Item | Source | Value |
| ----- | -------------- | ----- |
| Prescription Reference | (300a, 0016) Dose Reference Description | 'zz_6cm'|
| Reference Point Coordinates | (300a, 0018) Dose Reference Point Coordinates | [0, -40, 0] |
| Isocentre position | (300a, 012c) Isocenter Position | [0, -100, 0*] |
| Target Prescription Dose | (300a, 0026) Target Prescription Dose  | 36.0 |
| Target Prescription Units | Table C.8-46 | Gy |

For the specific fraction:

| Item | Source | Value |
| ----- | -------------- | ----- |
| Dose | (300a, 0084) Beam Dose | 3.0 |
| Beam Meterset (i.e. MU) | (300a, 0086) Beam Meterset | 367.670378557074 | 

For the specific beam:

| Item | Source | Value |
| ---- | ------ | ----- |
| Beam Limiting Device 1 | (300a, 00b8) RT Beam Limiting Device Type | X |
| Beam Limiting Device 1 - Leaf/Jaw Positions | (300a, 011c) Leaf/Jaw Positions | -35, 35 ** |
| Beam Limiting Device 2 | (300a, 00b8) RT Beam Limiting Device Type | Y |
| Beam Limiting Device 2 - Leaf/Jaw Positions | (300a, 011c) Leaf/Jaw Positions | -20, 20 ** |
| Energy | (300a, 0114) Nominal Beam Energy | 10 |
| Energy Unit | Table C.8-50 | MV |
| Source axis distance | (300a, 00b4) Source-Axis Distance | 1000.0 | 

\* It's not strictly 0 in the z direction, but it is essentially 0.,,,
\*\* For aymmetric fields, you may need to determine the coordinate system to know which beam limiting device side moves in which direction. For this case it doesn't really matter.

## Calculation
Now we just need to perform the calculation.

From our lectures we know for a 100cm FSD beam, we can reduce the manual calculation to  $MU = \frac{d X R}{PDD}$. We therefore need to work out d, R and PDD, where d is the dose per fraction, R is the relative output factor, and PDD is the percentage depth dose.

d is given from prescription dose for the fraction, and is 3 Gy.

To work out R and PDD we will use the tables included in the resources section, but we first need the equivalent square. From the beam limiting devices we can see that it is a 70mm by 40mm field, therefore using the equivalent square formula $(\frac{2 * a * b}{a + b})$, it's equivalent square will be:

In [40]:
(2 * 70 * 40)/(70 + 40)

50.90909090909091

51 x 51 mm, or 5.1 x 5.1 cm

The source axis distance (SAD)/source surface distance (SSD) is 1000mm, or 100cm, so our standard PDD tables will be fine with no corrections. For the PDDs, see [PDD pdf](./resources/PDDs.pdf), and for relative outputs, see [output factors pdf](./resources/outputfactors.pdf)

Together then, we know it is a 10MV beam at 100cm FSD, at 5.1cm equivalent square field. We therefore use the 10MV set of tables in the PDDs, and in the output factors. 

From the details pulled out above we can see the reference dose point coordinates is at [0, -40, 0] and the isocentre is (for all intents and purposes) at [0, -100, 0]. The difference is 60mm, which ties up with our dose reference point label of zz_6cm, therefore we need to get a PDD for 6cm deep.

Using the second table, for a depth of 6cm, we can pull out the 5cm equivalent square PDD as 87.0% and for 6cm as 87.2%. We could interpolate between the two, but it'll be very close to 87.0% so we can just use that for now.

From the output factors for 10MV, at 100cm SSD, we can see that for an equivalent square of 5cm the output factor is 106.9%, and for 6cm is 104.8%. We can therefore interpolate to get the factor for 5.1cm. We can use the formula $\frac{y-y_0}{x-x_0} = \frac{y_1-y_0}{x_1-x_0}$. Conveniently numpy has an interpolation function:

In [41]:
import numpy as np

In [42]:
np.interp(5.1,[5,6],[106.9,104.8])

106.69000000000001

So we have R as 106.7%, PDD as 87.0%, and d as 3 Gy. Together this gives an MU of:

In [43]:
(3*106.7)/87.0

3.6793103448275866

Comparing that against the meterset, we notice something wrong: we are a factor of 100 out! Remember back to lectures that we calibrate our outputs as 1MU/cGy! So if we instead convert to cGy first, we get:

In [44]:
(300*106.7)/87.0

367.9310344827586

What is the correct (appropriate?) number of significant figures for a meterset/number of MU? The linear accelerator can only count to one decimal place, so it makes sense to use that as our limiting factor.  

Therefore our calculation yields 367.9 MU - which is very close to the 367.7 in the plan object.

We can finally check that this value makes sense:
- the machine is calibrated to provide 1cGy/MU under reference conditions (10x10cm field, measured at depth of dose maximum, at 100cm SSD)
  - our field size is smaller, so less dose makes it way to the central axis, so our number of MU must go up
  - we are prescribing at a point deeper than the depth of dose maximum, as again the number of MU must go up.

## Assignment

### Part 1 - Calculation
In the resources folder, there is an RP object called `RP.singlefield-assignment.dcm`. 

1. Using this plan object, please produce a step-by-step calculation, and compare the calculated meterset with that in the plan object itself. [4 marks]
2. An OAR is present at 15cm deep from the surface of the patient, on the central axis - what dose would this organ at risk receive (assume it is a point i.e. has no volume) [2 marks]

Present your calculations step-by-step in a document. You may continue to write some code in this notebook to present the DICOM information, or you may download the DICOM file to open in your own environment.

### Part 2 - Single vs.  Multi-field Plans
In the resource folder, there is an RP object called `RP.multifield-assignment.dcm`.

3. Using this plan object, and comparing it against the singlefield assignment plan object, describe what differences are present between single and multifield plans. It may be useful to use a heirarchical diagram. [2 marks]

### Part 3 - Anonymisation
Considering the anonymisation of an RP object:
4. Select 2 DICOM tags that definitely require alteration, explain why, and what you would do to the data to make it anonymised [2 marks]
5. Select 2 DICOM tags that should not be altered and explain why [2 marks]
6. Select 2 DICOM tags that may or may be not altered, depending on the use of the anonymised dataset. Critically appraise the alteration of these, and describe some of the risks in retaining such information [3 marks]